# EXTRACT LOAD TRANSFORM (ETL)

## Objectives

In this project we will:

*   Run the ETL process
*   Extract bank and market cap data from the JSON file `bank_market_cap.json`
*   Transform the market cap currency using the exchange rate data
*   Load the transformed data into a seperate CSV


For this project, we are going to be using Python and several Python libraries. Some of these libraries might be installed in this notebook. The cells below will install these libraries when executed.


In [1]:
!pip install glob
!pip install pandas
!pip install requests
!pip install datetime

ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob
     |████████████████████████████████| 60 kB 11.0 MB/s eta 0:00:01
     |████████████████████████████████| 259 kB 46.4 MB/s eta 0:00:01


## Imports

Import any additional libraries you may need here.


In [2]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler.

In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2021-11-22 10:29:53--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2021-11-22 10:29:54 (87.7 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2021-11-22 10:29:55--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [4]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. we will Store the data in a `pandas` dataframe. we Use the following list for the columns.


In [5]:
columns=['Name','Market Cap (US$ Billion)']

In [8]:
def extract():
    # Write your code here
    extracted_data = pd.DataFrame(columns=columns)
    for jsonfile in glob.glob("*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    return extracted_data
    

We load the file <code>exchange_rates.csv</code> as a dataframe and find the exchange rate for British pounds with the symbol <code>GBP</code>, store it in the variable  <code>exchange_rate</code>, you will be asked for the number. Hint: set the parameter  <code>index_col</code> to 0.


In [28]:
df = pd.read_csv("exchange_rates.csv", index_col=False, names=['Currency', 'Rates'], header=None, skiprows=1)
GBP_index = df.index[df['Currency'] == "GBP"].tolist()
exchange_rate = df['Rates'][GBP_index].tolist()
exchange_rate

[0.7323984208000001]

## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1.  Changes the `Market Cap (US$ Billion)` column from USD to GBP
2.  Rounds the Market Cap (US$ Billion)\` column to 3 decimal places
3.  Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [11]:
def transform(data):
    data["Market Cap (US$ Billion)"]  = data["Market Cap (US$ Billion)"] * exchange_rate
    data["Market Cap (US$ Billion)"] = data["Market Cap (US$ Billion)"].round(3)
    data.rename(columns={"Market Cap (US$ Billion)": "Market Cap (GBP$ Billion)"}, inplace = True)
    return data

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [29]:
def load(data):
    data.to_csv('bank_market_cap_gbp.csv', index=False)

## Logging Function


Write the logging function <code>log</code> to log your data:


In [30]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')
    

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [31]:
log("ETL Job Started")
log("Extract phase Started")

### Extract


We use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [32]:
# Call the function here
extracted_data = extract()
# Print the rows here
extracted_data.head(5)

,Name,Market Cap (US$ Billion)
0,Banco Bilbao Vizcaya Argentaria,60.678
1,Japan Post Bank,60.525
2,The Bank of New York Mellon,59.823
3,Shanghai Pudong Development Bank,58.796
4,Industrial Bank (China),57.388


Log the data as <code>"Extract phase Ended"</code>


In [33]:
log("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [34]:
log("Transform phase Started")

We use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [35]:
# Call the function here
transform_data = transform(extracted_data)
# Print the first 5 rows here
transform_data.head(5)

,Name,Market Cap (GBP$ Billion)
0,Banco Bilbao Vizcaya Argentaria,44.440
1,Japan Post Bank,44.328
2,The Bank of New York Mellon,43.814
3,Shanghai Pudong Development Bank,43.062
4,Industrial Bank (China),42.031


Log your data <code>"Transform phase Ended"</code>


In [36]:
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [37]:
log("Load phase Started")

Call the load function


In [38]:
load(transform_data)

Log the following `"Load phase Ended"`.


In [39]:
log("Load phase Ended")

| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2021-11-25        | 0.1     | Robiu Aliu .O.    | Created ETL project                |